In [ ]:
# numpy downgrade가 필요할 수 있음
!python -m pip install numpy==1.2

In [1]:
!pip install ensemble_boxes

     |████████████████████████████████| 3.4 MB 24.4 MB/s eta 0:00:01
     |████████████████████████████████| 15.3 MB 75.1 MB/s eta 0:00:01
     |████████████████████████████████| 26.3 MB 72.6 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.2
    Uninstalling numpy-1.21.2:
      Successfully uninstalled numpy-1.21.2


In [9]:
import pandas as pd
from ensemble_boxes import *
import numpy as np
from pycocotools.coco import COCO

In [10]:
submission_df = pd.read_csv('./sample/submission_yolov5x.csv')
submission_df = submission_df.drop(['index'], axis=1)
submission_df.to_csv('submission_yolov5x_new.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: './sample/submission_yolov5x.csv'

In [40]:
# ensemble csv files
#submission_files = ['./sample_submission.csv'] # submission lists
submission_files = ['./sample/submission_cascade.csv', './sample/submission_cascade_swin.csv',
                    './sample/submission_cascade_swin_soft.csv', './sample/submission_Resnext101_32x4d_DetectoRS.csv',
                    './sample/submission_cascade_swin_panet.csv', './sample/submission_yolor_new.csv', './sample/submission_yolov5x_modified.csv',
                    './sample/wTTA.csv'] # submission lists
submission_df = [pd.read_csv(file) for file in submission_files]

In [42]:
submission_df

[                                       PredictionString       image_id
 0     0 0.14121126 482.51205 575.3071 583.17377 659....  test/0000.jpg
 1     0 0.4715332 646.6021 655.3318 678.696 831.3629...  test/0001.jpg
 2     0 0.36675873 884.0751 453.48718 1009.5719 634....  test/0002.jpg
 3     0 0.15888673 873.61426 303.20844 1021.6727 765...  test/0003.jpg
 4     0 0.25711563 228.34488 353.0314 864.78687 782....  test/0004.jpg
 ...                                                 ...            ...
 4866  0 0.6522544 294.53815 369.40198 663.5018 696.0...  test/4866.jpg
 4867  0 0.2682657 625.76434 646.56854 769.7504 735.3...  test/4867.jpg
 4868  0 0.11228676 466.5624 666.29456 538.986 704.84...  test/4868.jpg
 4869  0 0.8171457 9.350311 200.68442 1009.0178 934.9...  test/4869.jpg
 4870  0 0.3176533 98.74609 232.97247 212.50085 345.6...  test/4870.jpg
 
 [4871 rows x 2 columns],
                                        PredictionString       image_id
 0     0 0.7334854 483.01956 596.369

In [43]:
image_ids = submission_df[0]['image_id'].tolist()

In [44]:
annotation = '../dataset/test.json'
coco = COCO(annotation)

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


In [45]:
prediction_strings = []
file_names = []
iou_thr = 0.6

for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
    
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        
        if len(predict_list)==0 or len(predict_list)==1:
            continue
            
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
            
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
    
    if len(boxes_list):
        boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, iou_thr=iou_thr)
        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:73: UserWarning: X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:85: UserWarning: Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.47802734375, 0.0, 0.60009765625, 0.0].
  warnings.warn("Zero area box skipped: {}.".format(box_part))


In [46]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('submission_FINAL_ensemble_wbf.csv', index=False)

submission.head()
submission

,PredictionString,image_id
0,7 0.8832376599311829 602.7208251953125 504.265...,test/0000.jpg
1,5 0.6775245070457458 133.5726776123047 0.37436...,test/0001.jpg
2,1 0.6645795106887817 779.540771484375 416.3238...,test/0002.jpg
3,9 0.3936730921268463 150.1287078857422 268.769...,test/0003.jpg
4,1 0.5169661045074463 203.116455078125 319.1077...,test/0004.jpg
...,...,...
4866,0 0.5677744150161743 293.7481689453125 356.762...,test/4866.jpg
4867,3 0.6352837681770325 414.136962890625 320.4855...,test/4867.jpg
4868,7 0.6148608326911926 112.14129638671875 481.82...,test/4868.jpg
4869,0 0.5118367671966553 10.939021110534668 219.88...,test/4869.jpg
